In [418]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [419]:
filepath = '/content/drive/MyDrive/Progetto Colab/'

# Importazione e pre-process del corpus

In [420]:
import pandas as pd
df = pd.read_excel(filepath+'data/Finance.xlsx')
df.head()

,Tipo,Score,Text
0,Finance,Positive,'bout to pas in pps.. im in both '
1,Finance,Positive,'short toasting very well'
2,Finance,Negative,'is dead money for the rest of the year a ipho...
3,Finance,Positive,'already climbing pm.'
4,Finance,Positive,'how are yall bearish smfh'


In [421]:
df_not_na = df[~(df['Text'].isna())]
raw_text = df_not_na['Text']

In [422]:
import nltk
#tassonomia della lingua inglese
nltk.download('wordnet')
#parole di uso comune che non hanno significato, e, che , li, gli
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [423]:

from tqdm import tqdm
#riduzione delle parole a una radice, il lematizer scegli una forma base comune come il tempo essere, e tutti i verbi saranno messi come esso
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

#tutte le parole in minuscolo e sostituire [^\w\s\d], (elimina tutto ciò che non è parole, spazi, numeri) 
text = raw_text.str.lower().str.replace('[^\w\s\d]',' ', regex=True)
#separiamo con gli spazi
text = text.str.split()

#per ogni parola si applica il lemmataizer
text = text.apply(lambda x: [lemmatizer.lemmatize(sent) for sent in x])
#tolgo tutte le parole che hanno una lunghezza minore di 2
for row in text:
    [row.remove(i) for i in row if len(i) < 2]
text.head()

0                [bout, to, pa, in, pps, im, in, both]
1                        [short, toasting, very, well]
2    [is, dead, money, for, the, rest, of, the, yea...
3                              [already, climbing, pm]
4                      [how, are, yall, bearish, smfh]
Name: Text, dtype: object

In [424]:
from nltk.corpus import stopwords
#Parole da eliminare
stop = stopwords.words('english')

train_sentences = []
for row in text:
    train_sentences.append([item for item in row if item not in stop])

#FastText

In [425]:
from gensim.models import FastText

#min_n dimensione minima dei caratteri, considera parole di almeno di 2 lettere
#max_n dimensione massima dei caratteri
#stiamo considerando le parole da 2 a 6 lettere
#min_count numero minimo di parole


ft_model = FastText(sg=0, hs=1, sentences=train_sentences, size=100, window=5, min_count=5, iter=2, min_n=2, max_n=6) #gensim 3

#Caricamento dizionario

In [426]:
import json
tf = open(filepath+"dizionario/Dizionario_finance.json", "r")
new_words = json.load(tf)
print(new_words)

{'rose': 4, 'increase': 4, 'positive': 4, 'load': 4, 'doubled': 4, 'green': 4, 'vulture': 4, 'higher': 4, 'increased': 4, 'bear': 4, 'example': 4, 'grew': 4, 'squeeze': 4, 'having': 4, 'improved': 4, 'base': 4, 'nio': 4, 'uptrend': 4, 'dip': 4, 'rocket': 4, 'narrowed': 4, 'business': 4, 'spread': 4, 'buy': 4, 'call': 4, 'launch': 4, '2b': 4, 'favorite': 4, 'cover': 4, 'nicely': 4, 'everyday': 4, 'baba': 4, 'electric': 4, 'signed': 4, 'auto': 4, 'panic': 4, 'clothing': 4, 'live': 4, 'division': 4, 'solid': 4, 'consolidation': 4, 'leap': 4, 'scared': 4, 'planning': 4, 'sept': 4, 'folk': 4, 'hey': 4, 'action': 4, 'above': 4, 'spike': 4, 'run': 4, 'cnbc': 4, 'youre': 4, 'ill': 4, 'awarded': 4, 'stated': 4, 'recycling': 4, 'missed': 4, 'ol': 4, 'up': 4, '02': 4, 'cross': 4, 'agreement': 4, 'congrats': 4, 'maker': 4, 'we': 4, 'lunch': 4, '250': 4, 'covering': 4, 'rise': 4, 'video': 4, 'become': 4, 'plus': 4, 'aim': 4, 'upgrade': 4, 'understand': 4, 'region': 4, '46': 4, 'offer': 4, 'patient'

# Assegnazione pesi inserendo 4

In [427]:

diz={}
a=0
for i in list(new_words.keys()):
  for j in range(0,9):
    if ft_model.wv.most_similar(i)[j][1]>0.99: #Riferita a ogni parola del dizionario, si selezionata la più simile con punteggio maggiore di 0.99 e si inserisce 4
      diz[ft_model.wv.most_similar(i)[j][0]]=4
    elif ft_model.wv.most_similar(i)[j][1]>0.99 and new_words[i]==-4: #Riferita a ogni parola del dizionario, si selezionata la più simile con punteggio maggiore di 0.99 e si inserisce -4
      diz[ft_model.wv.most_similar(i)[j][0]]=-4
    else:
      break


# Assegnazione pesi aumentando di 1

## Caricamento dizionario vader

In [428]:
vader = pd.read_csv(filepath+'data/Vader.csv', delimiter=";",names=["nome","valore"])
d =  dict([([a,b]) for a,b in zip(vader['nome'], vader['valore'])])

In [429]:
'''
diz={}
for i in list(new_words.keys()):
  for j in range(0,9):

    if ft_model.wv.most_similar(i)[j][1]>0.99 and new_words[i]==4 and ft_model.wv.most_similar(i)[j][0] not in list(new_words.keys()) :
      if ft_model.wv.most_similar(i)[j][0] in list(d.keys()):
        diz[ft_model.wv.most_similar(i)[j][0]]=d[ft_model.wv.most_similar(i)[j][0]]+1
      else:
        diz[ft_model.wv.most_similar(i)[j][0]]=3.5

    elif ft_model.wv.most_similar(i)[j][1]>0.99 and new_words[i]==-4:
      if ft_model.wv.most_similar(i)[j][0] in list(d.keys()):
        diz[ft_model.wv.most_similar(i)[j][0]]=d[ft_model.wv.most_similar(i)[j][0]]-1
      else:
        diz[ft_model.wv.most_similar(i)[j][0]]=-3.5

    else:
      break
'''

'\ndiz={}\nfor i in list(new_words.keys()):\n  for j in range(0,9):\n\n    if ft_model.wv.most_similar(i)[j][1]>0.99 and new_words[i]==4 and ft_model.wv.most_similar(i)[j][0] not in list(new_words.keys()) :\n      if ft_model.wv.most_similar(i)[j][0] in list(d.keys()):\n        diz[ft_model.wv.most_similar(i)[j][0]]=d[ft_model.wv.most_similar(i)[j][0]]+1\n      else:\n        diz[ft_model.wv.most_similar(i)[j][0]]=3.5\n\n    elif ft_model.wv.most_similar(i)[j][1]>0.99 and new_words[i]==-4:\n      if ft_model.wv.most_similar(i)[j][0] in list(d.keys()):\n        diz[ft_model.wv.most_similar(i)[j][0]]=d[ft_model.wv.most_similar(i)[j][0]]-1\n      else:\n        diz[ft_model.wv.most_similar(i)[j][0]]=-3.5\n\n    else:\n      break\n'

# Aggiornamento dizionario e salvataggio

In [430]:
new_words.update(diz)
import json
tf = open(filepath+"dizionario/Dizionario_finance.json", "w")
json.dump(new_words,tf)
tf.close()